In [ ]:
!git clone https://github.com/tGhattas/bamba.git

In [ ]:
%cd bamba

In [ ]:
!git pull
!pip install . -q

In [ ]:
!nvidia-smi

In [ ]:
!pwd

In [ ]:
# prompt: authenticate hugging face

!pip install huggingface_hub -q

from huggingface_hub import notebook_login

notebook_login()


In [ ]:
!pip3 install causal-conv1d>=1.2.0 -q
!pip3 install datasets -q
# !pip install . -q
# !pip install zstandard -q
!pip3 install torch -q
!pip3 install tqdm -q
!pip3 install wandb -q
!pip3 install transformers #-q



In [ ]:
#wandb login
!pip3 install --upgrade transformers -q


In [ ]:
import importlib
import  evals.distillation as dm
# importlib.reload(dm)

In [ ]:
import torch
"cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
dm.train(limit=5000)

In [1]:
import torch
import torch.nn as nn
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from torch.utils.data import DataLoader
from itertools import islice
# from mamba_ssm.models.mixer_seq_simple import MambaLMHeadModel, MambaConfig
from tqdm import tqdm
HF_PADDING_IGNORE = -100

# teacher_model_path = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"


/Users/tamer/miniconda3/envs/myvenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
dataset = load_dataset("monology/pile-uncopyrighted", streaming=True)


# Load the teacher tokenizer
teacher_tokenizer = AutoTokenizer.from_pretrained(teacher_model_path)

# Tokenize the dataset
def tokenize_function(examples):
    return teacher_tokenizer(examples["text"], truncation=True, padding="max_length", max_length=1024, return_tensors="pt")

tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# Create the data loader
data_loader = DataLoader(tokenized_datasets["train"], batch_size=4, num_workers=4)


In [ ]:
for i in tokenized_datasets["train"]:
    print(i)
    break

In [ ]:
for i in tokenized_datasets["train"]:
    print(teacher_tokenizer.decode(i["input_ids"]))
    break

In [ ]:
from itertools import islice

for i in islice(tokenized_datasets["train"],50):
    print(i["input_ids"])
    print(i["input_ids"].shape)
    print(i["attention_mask"])
    

In [ ]:
data_loader = DataLoader(tokenized_datasets["train"], batch_size=4, num_workers=4)

In [ ]:
for batch in islice(data_loader, 10):
    print(batch)
    break

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig



teacher_tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")
student_tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-790m-hf")
teacher_vocab = set(teacher_tokenizer.get_vocab())
student_vocab = set(student_tokenizer.get_vocab())
student_tokenizer.add_tokens(list(teacher_vocab - student_vocab))
tokens_to_remove = list(student_vocab - teacher_vocab)
new_student_vocab = student_tokenizer.get_vocab()
for token in tokens_to_remove:
    del new_student_vocab[token]


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [19]:
class ModifiedMambaTokenizerFactory:
    def __init__(self, student_tokenizer: AutoTokenizer, teacher_tokenizer: AutoTokenizer):
        self.student_tokenizer = student_tokenizer
        self.teacher_tokenizer = teacher_tokenizer
        teacher_vocab = set(teacher_tokenizer.get_vocab())
        student_vocab = set(student_tokenizer.get_vocab())
        student_tokenizer.add_tokens(list(teacher_vocab - student_vocab)[:len(teacher_vocab) - len(student_vocab)]) #
    

    def get_modified_tokenizer(self):
        return self.student_tokenizer

In [20]:
teacher_tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")
student_tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-790m-hf")
m = ModifiedMambaTokenizerFactory(student_tokenizer, teacher_tokenizer)
student_tokenizer = m.get_modified_tokenizer()
print(student_tokenizer.get_vocab())

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


{'Ġí': 25811, 'Ġglancing': 48147, 'Ġintuit': 72790, 'ï¼īãģ®': 126507, 'Ġ************************************************': 92466, 'Ġìłķìĭł': 65529, 'Ġeinige': 85988, 'Ð²Ð°Ð²ÑģÑı': 54663, 'Ġdar': 13681, 'Ġfind': 1089, 'ĠRiding': 66851, '_sn': 112134, 'decrypt': 107707, 'Ġ=================================================': 111354, 'gee': 104313, '.isDebugEnabled': 107475, 'Trees': 67865, '(te': 68622, 'vertical': 26806, 'é«ĺä¸Ń': 83281, '[];Ċ': 126838, 'ĠâĮĴ': 101955, 'privation': 63642, 'Ġclassics': 42975, 'ĠGL': 9653, '.maximum': 56771, 'ç«ŀ': 94967, 'away': 12594, 'Ġthá»«a': 80357, 'Ġparasitic': 38167, 'ì¡±': 54926, 'developed': 35208, 'Ġconductors': 39713, 'Ġpropriet': 41473, 'ĠHermes': 93641, 'spa': 90329, 'Ġ-=': 24869, 'getMethod': 123028, 'enty': 4108, 'rett': 12436, "'er": 112021, 'vals': 9863, 'propylene': 49848, '.Nullable': 87864, 'ĠBureau': 15904, 'Ev': 12018, 'Cache': 15554, 'Ġunethical': 77407, 'interrupt': 30933, 'ç¦ģ': 122372, 'Vertex': 27384, 'Ġrepresented': 6607, 'ĠTaip

In [22]:
len(student_tokenizer.get_vocab()), len(teacher_tokenizer.get_vocab())

(128256, 128256)

In [ ]:
print(len(teacher_tokenizer.get_vocab()), len(student_tokenizer.get_vocab()))
print(len(tokens_to_remove))
print(len(new_student_vocab))


In [ ]:
new_vocab = {k: v for k, v in sorted(new_student_vocab.items(), key=lambda item: item[1])}

In [18]:
print(len(student_tokenizer.get_vocab()), len(new_student_vocab))
student_tokenizer.save_pretrained("my/state-spaces/mamba-790m-hf")
student_tokenizer = AutoTokenizer.from_pretrained("my/state-spaces/mamba-790m-hf")
print(len(student_tokenizer.get_vocab()))

136486 128256


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


136486


In [15]:
student_tokenizer.

AttributeError: can't set attribute 'vocab'

In [3]:
model = AutoModelForCausalLM.from_pretrained("state-spaces/mamba-370m-hf")

The fast path is not available because on of `(selective_state_update, selective_scan_fn, causal_conv1d_fn, causal_conv1d_update, mamba_inner_fn)` is None. Falling back to the naive implementation. To install follow https://github.com/state-spaces/mamba/#installation and https://github.com/Dao-AILab/causal-conv1d


In [25]:
model#.resize_token_embeddings(len(student_tokenizer))

MambaForCausalLM(
  (backbone): MambaModel(
    (embeddings): Embedding(128256, 1024)
    (layers): ModuleList(
      (0-47): 48 x MambaBlock(
        (norm): MambaRMSNorm()
        (mixer): MambaMixer(
          (conv1d): Conv1d(2048, 2048, kernel_size=(4,), stride=(1,), padding=(3,), groups=2048)
          (act): SiLU()
          (in_proj): Linear(in_features=1024, out_features=4096, bias=False)
          (x_proj): Linear(in_features=2048, out_features=96, bias=False)
          (dt_proj): Linear(in_features=64, out_features=2048, bias=True)
          (out_proj): Linear(in_features=2048, out_features=1024, bias=False)
        )
      )
    )
    (norm_f): MambaRMSNorm()
  )
  (lm_head): Linear(in_features=1024, out_features=128256, bias=False)
)